In [1]:
import urllib.parse
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd
import time
import logging
from bs4 import BeautifulSoup
import re

In [2]:
# Fungsi untuk konfigurasi driver
def configure_driver():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36")
    chrome_options.add_argument('--disable-dev-shm-usage')  
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--remote-debugging-port=9222')

    # Tentukan path ke chromedriver
    service = Service(r'chromedriver-win64\chromedriver.exe')
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

In [ ]:
# Fungsi untuk mengklik tombol "next" dan mengambil data
def scrape_next_page(driver, data, last_data):
    # Ambil HTML dari halaman yang sedang dimuat
    html_content = driver.page_source

    # Parse HTML dengan BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Menyimpan HTML yang terstruktur (rapi) ke dalam file
    with open('page_source_idx.html', 'w', encoding='utf-8') as file:
        file.write(soup.prettify())

    # Menemukan semua baris di dalam tabel
    rows = soup.find_all('tr')

    # List untuk menyimpan data yang ditemukan di halaman ini
    current_data = []

    # Iterasi untuk mencari data yang relevan
    for row in rows:
        # Mencari kode perusahaan
        kode_perusahaan = row.find('td', class_='vgt-left-align')
        if kode_perusahaan:
            kode = kode_perusahaan.get_text(strip=True)

            # Mencari nama perusahaan dan link
            nama_perusahaan = row.find('a')
            if nama_perusahaan:
                nama = nama_perusahaan.get_text(strip=True)
                link = "https://www.idx.co.id" + nama_perusahaan.get('href')

            # Mencari tanggal pencatatan
            tanggal_pencatatan = row.find_all('td', class_='vgt-left-align')
            if len(tanggal_pencatatan) > 2:
                tanggal = tanggal_pencatatan[2].get_text(strip=True)

                # Menyimpan data ke dalam list
                current_data.append({
                    'Kode': kode,
                    'Nama': nama,
                    'Link': link,
                    'Tanggal Pencatatan': tanggal
                })

    # Cek apakah data saat ini sama dengan data sebelumnya (halaman baru)
    if current_data == last_data:
        print("Data tidak berubah, halaman tidak bergerak.")
        return False, current_data  # Jika data tidak berubah, return False

    # Menambahkan data baru yang ditemukan
    data.extend(current_data)

    # Klik tombol "next" jika tidak ada kelas "disabled"
    try:
        # Memilih tombol "next" yang benar dengan chevron right dan tidak ada kelas disabled
        next_button = driver.find_element(By.XPATH, "//button[contains(@class, 'footer__navigation__page-btn') and not(contains(@class, 'disabled')) and span[contains(@aria-hidden, 'true') and contains(@class, 'chevron right')]]")
        next_button.click()  # Klik tombol "next"
        time.sleep(3)  # Tunggu beberapa detik agar halaman dimuat
    except Exception as e:
        print("Tidak dapat menemukan tombol next:", e)
        return False, current_data  # Jika tombol tidak ditemukan, return False

    return True, current_data  # Jika berhasil klik tombol "next"

# Setup WebDriver
driver = configure_driver()

# Akses halaman web
driver.get("https://www.idx.co.id/id/perusahaan-tercatat/profil-perusahaan-tercatat")

# Tunggu beberapa detik agar halaman sepenuhnya dimuat
time.sleep(5)

# List untuk menyimpan data
data = []
last_data = []  # Menyimpan data halaman sebelumnya

# Lakukan scraping selama tombol "next" tersedia
while True:
    should_continue, last_data = scrape_next_page(driver, data, last_data)
    if not should_continue:
        break  # Berhenti jika data tidak berubah atau tombol "next" tidak ditemukan

# Mengonversi data ke dalam DataFrame
df_idx = pd.DataFrame(data)

# Quit driver
driver.quit()

In [ ]:
df_idx

,Kode,Nama,Link,Tanggal Pencatatan
0,AADI,PT Adaro Andalan Indonesia Tbk,https://www.idx.co.id/id/perusahaan-tercatat/p...,05 Des 2024
1,AALI,Astra Agro Lestari Tbk,https://www.idx.co.id/id/perusahaan-tercatat/p...,09 Des 1997
2,ABBA,Mahaka Media Tbk,https://www.idx.co.id/id/perusahaan-tercatat/p...,03 Apr 2002
3,ABDA,Asuransi Bina Dana Arta Tbk,https://www.idx.co.id/id/perusahaan-tercatat/p...,06 Jul 1989
4,ABMM,ABM Investama Tbk,https://www.idx.co.id/id/perusahaan-tercatat/p...,06 Des 2011
...,...,...,...,...
946,ZATA,PT Bersama Zatta Jaya Tbk,https://www.idx.co.id/id/perusahaan-tercatat/p...,10 Nov 2022
947,ZBRA,PT Dosni Roha Indonesia Tbk,https://www.idx.co.id/id/perusahaan-tercatat/p...,01 Agt 1991
948,ZINC,PT Kapuas Prima Coal Tbk,https://www.idx.co.id/id/perusahaan-tercatat/p...,16 Okt 2017
949,ZONE,PT Mega Perintis Tbk.,https://www.idx.co.id/id/perusahaan-tercatat/p...,12 Des 2018


In [ ]:
# df_idx.to_csv('idx.csv', index=False)
# df_idx = pd.read_csv('idx.csv')

In [17]:
df_idx.rename(columns={'Nama': 'Nama Perusahaan', 'Kode':'Kode Perusahaan'}, inplace=True)

In [84]:
urls = df_idx["Link"].tolist()

In [85]:
# Setup WebDriver
driver = configure_driver()

# Function to extract company data from the page
def extract_company_data(url):
    driver.get(url)
    time.sleep(1.5)  # Wait for the page to load completely

    # Get page source
    html_content = driver.page_source
    
    # Parse the page with BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Extract the company details
    company_data = {}
    
    # Find the table body and extract each row's data
    rows = soup.find_all('tr', {'data-v-f28e1a12': True})
    
    for row in rows:
        columns = row.find_all('td')
        if len(columns) >= 3:
            label = columns[0].text.strip()
            value = columns[2].text.strip()

            # Map the labels to specific company data keys
            if label == 'Nama':
                company_data['Nama'] = value
            elif label == 'Kode':
                company_data['Kode'] = value
            elif label == 'Alamat Kantor':
                company_data['Alamat Kantor'] = value
            elif label == 'Alamat Email':
                company_data['Alamat Email'] = value
            elif label == 'Telepon':
                company_data['Telepon'] = value
            elif label == 'Fax':
                company_data['Fax'] = value
            elif label == 'NPWP':
                company_data['NPWP'] = value
            elif label == 'Situs':
                company_data['Situs'] = value
            elif label == 'Tanggal Pencatatan':
                company_data['Tanggal Pencatatan'] = value
            elif label == 'Papan Pencatatan':
                company_data['Papan Pencatatan'] = value
            elif label == 'Bidang Usaha Utama':
                company_data['Bidang Usaha Utama'] = value
            elif label == 'Sektor':
                company_data['Sektor'] = value
            elif label == 'Subsektor':
                company_data['Subsektor'] = value
            elif label == 'Industri':
                company_data['Industri'] = value
            elif label == 'Subindustri':
                company_data['Subindustri'] = value
            elif label == 'Biro Administrasi Efek':
                company_data['Biro Administrasi Efek'] = value
    
    # Extract "Sekretaris Perusahaan" details
    sek_perusahaan_section = soup.find('span', {'id': 'Secretary'})
    if sek_perusahaan_section:
        # Find all rows within the Sekretaris Perusahaan section
        sek_rows = sek_perusahaan_section.find_next('table').find_all('tr')
        
        for sek_row in sek_rows:
            sek_columns = sek_row.find_all('td')
            if len(sek_columns) >= 3:
                label = sek_columns[0].text.strip()
                value = sek_columns[2].text.strip()

                if label == 'Nama':
                    company_data['Sekretaris Nama'] = value
                elif label == 'Nomor Telepon':
                    company_data['Sekretaris Telepon'] = value

    return company_data

# Iterate over all URLs and extract the data
all_company_data = []
for url in urls:
    company_data = extract_company_data(url)
    all_company_data.append(company_data)

# Close the browser
driver.quit()

# Create DataFrame
df_company_detail = pd.DataFrame(all_company_data)

In [ ]:
# df_company_detail.to_csv('company_data_idx.csv', index=False)

In [86]:
df_company_detail

,Nama,Kode,Alamat Kantor,Alamat Email,Telepon,Fax,NPWP,Situs,Tanggal Pencatatan,Papan Pencatatan,Bidang Usaha Utama,Sektor,Subsektor,Industri,Subindustri,Biro Administrasi Efek,Sekretaris Nama,Sekretaris Telepon
0,Ray Aryaputra,AADI,Cyber 2 Tower Lantai 26\nJl. H.R. Rasuna Said ...,corsec@adaroindonesia.com,(021) 2553 3065,(021) 2553 3066,02.433.115.9-091.000,www.adaroindonesia.com,2024-12-05,Utama,Perusahaan holding (yang menaungi Perusahaan A...,Energi,"Minyak, Gas & Batu Bara",Batu Bara,Produksi Batu Bara,PT. Datindo Entrycom,Ray Aryaputra,(021) 2553 3065
1,Tingning Sukowignjo,AALI,Jl. Puloayang Raya Kawasan Industri Pulo Gadun...,investor@astra-agro.co.id,461-65-55,"461-6655, 461-6677, 461-6688",001.334.427.0-054.000,http://www.astra-agro.co.id,1997-12-09,Utama,Agriculture Plantation,Barang Konsumen Primer,Makanan & Minuman,Produk Makanan Pertanian,Perkebunan & Tanaman Pangan,PT. Raya Saham Registra,Tingning Sukowignjo,021-4616555
2,S. Pramudityo Anggoro,ABBA,"Sahid Sudirman Centre Lt. 10, Jl. Jend. Sudirm...",corsec@mahakamedia.com,(021) 573 9203,(021) 573 9210,01.609.052.4-054.000,www.mahakax.com,2002-04-03,Pemantauan Khusus,Media dan Percetakan,Barang Konsumen Non-Primer,Media & Hiburan,Media,Penerbitan,PT. Adimitra Jasa Korpora,S. Pramudityo Anggoro,021-5739203
3,Fausia Isti Tanoso,ABDA,Plaza Asia Lt. 27 Jl. Jend. Sudirman Kav. 59 J...,fausia.tanoso@oona-insurance.co.id,5140 1688,5140 1698,01.362.014.1-054.000,www.myoona.id,1989-07-06,Pemantauan Khusus,Asuransi Umum,Keuangan,Asuransi,Asuransi,Asuransi Umum,PT. Raya Saham Registra,Fausia Isti Tanoso,021-51401688
4,Hans Christian Manoe,ABMM,Gedung TMT 1 Lantai 18 Jl. Cilandak KKO No. 1 ...,corporate.secretary@abm-investama.com,021-29976767,021-29976768,02.504.191.4-054.000,www.abm-investama.com,2011-12-06,Utama,Aktivitas konsultasi manajemen dan lainnya ser...,Energi,"Minyak, Gas & Batu Bara",Batu Bara,Produksi Batu Bara,PT. Datindo Entrycom,Hans Christian Manoe,021-29976767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946,Irvan Rachmawan,ZATA,"Komplek Industri Prapanca No.24, Cigondewah Ka...",irvan@elcorps.com,(022) 86017900,,31.569.863.9-428.000,www.elcorps.com,2022-11-10,Pemantauan Khusus,Aktivitas Perusahaan Holding dan aktivitas Usa...,Barang Konsumen Non-Primer,Perdagangan Ritel,Ritel Khusus,Ritel Pakaian & Tekstil,PT. Adimitra Jasa Korpora,Irvan Rachmawan,+62 82115558881
947,Ida Widayani,ZBRA,"Graha Dosni Roha, Jl Budi Raya No. 9B, Kemangg...",ida.widayani@dnr.id,021-530 5600,-,01.451.869.0-054.000,www.dnr.id,1991-08-01,Pengembangan,Perusahaan Holding dan Angkutan Bermotor untuk...,Perindustrian,Perusahaan Holding Multi Sektor,Perusahaan Holding Multi-sektor,Perusahaan Holding Multi-sektor,PT BSR Indonesia,Ida Widayani,021-27883900
948,Lucky Tajo,ZINC,"Jl. Pantai Indah Selatan I, Elang Laut Blok A ...",corsec@ptkpc.com,021 - 29676236,021 - 29676234,02.386.972.0-091.000,www.ptkpc.com,2017-10-16,Pemantauan Khusus,Pertambangan dan Perdagangan,Barang Baku,Barang Baku,Logam & Mineral,Logam & Mineral Lainnya,-,Lucky Tajo,021 29676236
949,Luki Rusli,ZONE,"Jalan Karet Pedurenan No. 240, Karet Kuningan,...","corpsec@megaperintis.co.id, luki.rusli@megaper...",(021) 5733 888; (021) 5290 4379,(021) 5290 5103,02.433.917.8-011.000,www.megaperintis.co.id,2018-12-12,Pengembangan,Perdagangan retail,Barang Konsumen Non-Primer,Perdagangan Ritel,Ritel Khusus,Ritel Pakaian & Tekstil,PT. Bima Registra,Luki Rusli,021-5733 888


In [19]:
# Menggabungkan kedua DataFrame berdasarkan baris
df_combined = pd.concat([df_idx, df_company_detail], axis=1)

In [22]:
df_combined.drop(columns=['Nama', 'Kode'], inplace=True)

In [23]:
# Menampilkan hasil gabungan untuk memastikan
display(df_combined.columns)
df_combined.head()  # Menampilkan 5 baris pertama dari df_combined

Index(['Kode Perusahaan', 'Nama Perusahaan', 'Link', 'Tanggal Pencatatan',
       'Alamat Kantor', 'Alamat Email', 'Telepon', 'Fax', 'NPWP', 'Situs',
       'Tanggal Pencatatan', 'Papan Pencatatan', 'Bidang Usaha Utama',
       'Sektor', 'Subsektor', 'Industri', 'Subindustri',
       'Biro Administrasi Efek', 'Sekretaris Nama', 'Sekretaris Telepon'],
      dtype='object')

,Kode Perusahaan,Nama Perusahaan,Link,Tanggal Pencatatan,Alamat Kantor,Alamat Email,Telepon,Fax,NPWP,Situs,Tanggal Pencatatan,Papan Pencatatan,Bidang Usaha Utama,Sektor,Subsektor,Industri,Subindustri,Biro Administrasi Efek,Sekretaris Nama,Sekretaris Telepon
0,AADI,PT Adaro Andalan Indonesia Tbk,https://www.idx.co.id/id/perusahaan-tercatat/p...,05 Des 2024,Cyber 2 Tower Lantai 26\nJl. H.R. Rasuna Said ...,corsec@adaroindonesia.com,(021) 2553 3065,(021) 2553 3066,02.433.115.9-091.000,www.adaroindonesia.com,2024-12-05,Utama,Perusahaan holding (yang menaungi Perusahaan A...,Energi,"Minyak, Gas & Batu Bara",Batu Bara,Produksi Batu Bara,PT. Datindo Entrycom,Ray Aryaputra,(021) 2553 3065
1,AALI,Astra Agro Lestari Tbk,https://www.idx.co.id/id/perusahaan-tercatat/p...,09 Des 1997,Jl. Puloayang Raya Kawasan Industri Pulo Gadun...,investor@astra-agro.co.id,461-65-55,"461-6655, 461-6677, 461-6688",001.334.427.0-054.000,http://www.astra-agro.co.id,1997-12-09,Utama,Agriculture Plantation,Barang Konsumen Primer,Makanan & Minuman,Produk Makanan Pertanian,Perkebunan & Tanaman Pangan,PT. Raya Saham Registra,Tingning Sukowignjo,021-4616555
2,ABBA,Mahaka Media Tbk,https://www.idx.co.id/id/perusahaan-tercatat/p...,03 Apr 2002,"Sahid Sudirman Centre Lt. 10, Jl. Jend. Sudirm...",corsec@mahakamedia.com,(021) 573 9203,(021) 573 9210,01.609.052.4-054.000,www.mahakax.com,2002-04-03,Pemantauan Khusus,Media dan Percetakan,Barang Konsumen Non-Primer,Media & Hiburan,Media,Penerbitan,PT. Adimitra Jasa Korpora,S. Pramudityo Anggoro,021-5739203
3,ABDA,Asuransi Bina Dana Arta Tbk,https://www.idx.co.id/id/perusahaan-tercatat/p...,06 Jul 1989,Plaza Asia Lt. 27 Jl. Jend. Sudirman Kav. 59 J...,fausia.tanoso@oona-insurance.co.id,5140 1688,5140 1698,01.362.014.1-054.000,www.myoona.id,1989-07-06,Pemantauan Khusus,Asuransi Umum,Keuangan,Asuransi,Asuransi,Asuransi Umum,PT. Raya Saham Registra,Fausia Isti Tanoso,021-51401688
4,ABMM,ABM Investama Tbk,https://www.idx.co.id/id/perusahaan-tercatat/p...,06 Des 2011,Gedung TMT 1 Lantai 18 Jl. Cilandak KKO No. 1 ...,corporate.secretary@abm-investama.com,021-29976767,021-29976768,02.504.191.4-054.000,www.abm-investama.com,2011-12-06,Utama,Aktivitas konsultasi manajemen dan lainnya ser...,Energi,"Minyak, Gas & Batu Bara",Batu Bara,Produksi Batu Bara,PT. Datindo Entrycom,Hans Christian Manoe,021-29976767


In [24]:
df_combined.to_excel('IDX-Bursa-Efek-Scrapper.xlsx', index=False)